# Генеративные модели для табличных временных рядов

Цели:

* познакомиться с "первым поколением" табличных моделей

Содержание:

* Классификация временных рядов
* DoppelGANger
  * Архитектура
* TimeGAN
* TimeGCI
* TsT-GAN


## Классификация временных рядов

По периодичности точек:
- регулярные - раз в сутки, минуту, месяц и т.п.
- нерегулярные - произвольные timestamp событий

По количеству измерений на точку:

- одномерные
- многомерные

Выделяют также табличные временные ряды (Tabular Time Series), с дополнительными категориальными атрибутами.

Причём категориальные атрибуты могут быть статическими, которые не меняются на протяжении всего ряда (например соц-дем пользователя в последовательности посещений страниц), так и динамическими, когда значение разное для каждой точки.


## DoppelGANger

[Generating High-fidelity, Synthetic Time Series Datasets with DoppelGANger, 2019](https://arxiv.org/pdf/1909.13403v1).

Есть реализация от автора (и не только), но отдельно стоит отметить оптимизированную версию в библиотеке gretel-synthetics.

В работе сравнивается с ранее использовавшимися вариантами для моделирования временных рядов:

* марковские процессы
* авторегрессионные модели (ARIMA с нейросетями и вот это всё)
* RNN
* обычные GAN

DoppelGANger состоит из двух GAN - основной и вспомогательной и работает с многомерными временными рядами, включающими как непрерывные, так и дискретные (категориальные) атрибуты.

Три типа атрибутов:

- timestamp
- статические (например, соц-дем переменные)
- динамические (например, длительность контакта)

Моделирование смешанных (дискретных и непрерывных) типов данных как известно непростая задача. DoppelGANger использует условную архитектуру, которая изолирует генерацию дискретных атрибутов от временных рядов, но использует их как условие при генерации временных рядов.

Ключевые особенности модели:

1. Раздельная генерация атрибутов: DoppelGANger разделяет генерацию статических атрибутов от временных рядов и подает атрибуты как condition на генератор временных рядов на каждом временном шаге.
2. Пакетная генерация: За один раз генерируется сразу несколько точек ряда.
3. Раздельная (decoupled) нормализация: Отдельно происходит генерация нормализованных временных рядов и реалистичных максимальных и минимальных значений, при условии на атрибуты.






### Архитектура

<figure>
<img src="https://raw.githubusercontent.com/sswt/dive2gai/w14/.github/images/w14/dgan_arch.png" alt="DGAN arch" width="60%">
<figcaption align = "center"> Image credit: DoppelGANger paper</figcaption>
</figure>

Генератор состоит из двух MLP (одна для атрибутов, другая для минимальных и максимальных значений) и RNN для временных рядов.

Дискриминатор - два MLP, основной для оценки правдоподобности всего сгенерированного ряда и вспомогательный только для атрибутов и min/max.

Для учёта корреляций между признаками и атрибутами генерация объектов данных разбивается на две фазы: сначала генерируются атрибуты, затем признаки, при условии на атрибуты.

Для генерации атрибутов используется сеть MLP, которая отображает входной вектор шума на атрибут $A_i$. Для генерации вещественных признаков используется архитектура на основе RNN. Для учёта корреляции между атрибутами и временными рядами, сгенерированный $A_i$ подаётся в качестве входа в RNN на каждом шаге.

Даже с RNN-генератором GAN все еще сложно уловить временные корреляции для длинных временных рядов (например 500). На выходе RNN стоит MLP, который  генерирует сразу несколько точек (например 10, тогда будет 50x10).

Этот выбор дизайна разделяет сложную задачу на две более простые, каждая из которых решается соответствующей архитектурой сети.

Для чего нужен Min/Max генератор? Разные признаки могут иметь разный масштаб и отличаться на несколько порядков, например в данных по сети полоса пропускания dial-up 1 - 10 Kb, а для оптоволокна 0.1 - 1 Gb. Это может приводить к коллапсу, от которого не спасёт и Wasserstein loss. Идея в том, чтобы вычислять среднюю точку и половину диапазона разброса (эквивалентно минимуму и максимуму) для каждого признака и генерировать эти атрибуты отдельно. А RNN работает уже с нормализованными атрибутами, которые после генерации приводятся к исходному масштабу.

Функция потерь - сумма WGAN-GP от двух дискриминаторов (тот, что по атрибутам идёт с коэффициентом $α$). Дискриминатор - MLP, а не RNN из-за Wasserstein loss.






Эксперименты в числе прочего проводились на [датасете из соревнования kaggle](https://www.kaggle.com/c/web-traffic-time-series-forecasting) по предсказанию количества посетителей страниц wikipedia.  Дискретность - сутки, данные starting from July, 1st, 2015 up until December 31st, 2016.

По сравнению с другими моделями, DGAN удалось лучше всех смоделировать сезонность. Это видно на графике автокорреляции ниже, где маленькие пики соответствуют недельной сезонности, а повышение в районе 365 дней - годовой. Из остальных моделей только RNN справилась с недельной сезонностью, а с годовой больше никто.

<figure>
<img src="https://raw.githubusercontent.com/sswt/dive2gai/w14/.github/images/w14/dgan_autocorr.png" alt="" width="60%">
<figcaption align = "center"> Image credit: DoppelGANger paper</figcaption>
</figure>


**NOTE**: In our work, we treat the timestamps as equally spaced and hence do not generate them explicitly. However we can easily extend this to unequally spaced timestamps by treating time as a continuous feature and generating inter-arrival times along with other features.


## TimeGAN

[Time-series Generative Adversarial Networks, 2019](https://proceedings.neurips.cc/paper/2019/file/c9efe5f26cd17ba6216bbe2a7d26d490-Paper.pdf), работа из Van der Schaar Lab (библиотека SynthCity).

TimeGAN - для учёта временных взаимосвязей, характерных для временных рядов. При этом временные ряды также со статическими атрибутами.

Два компонента - автоэнкодер и состязательный (GAN), которые обучаются одновременно. Четыре сетки - проекция в латентное пространство, восстановление из него, генератор и дискриминатор (каждая в двух ипостасях - для временных и статических фичей). Три лосса - классический бинарный состязательный, восстановления автоэнкодером и состязательный в пространстве эмбеддингов.

Как и в Professor forcing управление процессом генерации происходит в латентном пространстве, чтобы не мучаться с категориальными атрибутами.

Авторы TimeGAN отмечают, что методы Scheduled sampling, Professor forcing и Actor-critic имеют одну и ту же мотивацию для компенсации проблемы авторегрессионного MLE в RNN (exposure bias or closed-loop training vs open-loop inference), но они по своей природе детерминистические, в то время как TimeGAN имеет цель генерации данных и должен явно сэмплировать из выученного распределения.

5 лет назад многие боролись с проблемой exposure bias, но потом пришёл ChatGPT и всё отменил.

TimeGAN - модель, объединяющая темы исследования авторегрессионных моделей для предсказания последовательностей, методов генерации последовательностей на основе GAN и representation learning для временных рядов.

TimeGAN состоит из четырех компонентов:

* embedding-функция - две RNN для статических и временных признаков
* функция восстановления - FFN на каждом шаге
* генератор последовательности - две RNN для статических и временных признаков
* дискриминатор последовательности - двунаправленная RNN с FFN-выходным слоем.

Ключевым моментом является то, что компоненты автоэнкодера (первые два) обучаются совместно с adversarial-компонентами (последние два), так что TimeGAN одновременно учится кодировать признаки, генерировать представления и выполнять итерации во времени. Сеть эмбеддингов предоставляет латентное пространство, состязательная сеть работает в этом пространстве, а латентная динамика реальных и синтетических данных синхронизируется с помощью supervised loss.

Функции embedding и восстановления обеспечивают отображение между пространством признаков и латентным пространством, позволяя состязательной сети изучать основную временную динамику данных с помощью низкоразмерных представлений.

<figure>
<img src="https://raw.githubusercontent.com/sswt/dive2gai/w14/.github/images/w14/timegan.png" alt="TimeGAN arch" width="75%">
<figcaption align = "center"> Image credit: TimeGAN paper</figcaption>
</figure>


С помощью обучаемого пространства эмбеддингов, совместно оптимизируемого с целевой функциями состязательной компоненты (unsupervised) и предсказания следующей точки в латентном пространстве (supervised loss), сеть заставляют придерживаться динамики обучающих данных во время сэмплирования.

* Во-первых, в дополнение к состязательной функции потерь на реальных и синтетических последовательностях, вводится пошаговый supervised loss, использующий исходные данные в качестве supervision-сигнала, тем самым явно поощряя модель выучивать пошаговые условные распределения в данных.
* Во-вторых, вводится сеть эмбеддинга, чтобы обеспечить обратимое отображение между признаками и скрытыми представлениями, тем самым уменьшая высокую размерность пространства состязательного обучения. Это позволяет использовать тот факт, что временная динамика даже сложных систем часто определяется меньшим числом и меньшей размерностью факторов вариации. Важно отметить, что supervised loss минимизируется за счет совместного обучения сетей эмбеддинга и генератора, так что латентное пространство не только служит для повышения эффективности параметров, но и специально обусловлено для облегчения задачи выучивания временных связей для генератора.
* Наконец, схема работы обобщается на случай смешанных данных, когда одновременно генерируются статические данные и данные временных рядов.


Objectives:

- representation loss (LR) - точность восстановления из латентного пространства
- unsupervised loss (LU) - классификация реальных и сгененированных данных
- supervised loss (LS) - L2-norm для следующего t-го латентного вектора при данной последовательности эмбеддингов для 1..t-1 vectors (бинарной обратной связи дискриминатора может оказаться недостаточно для генератора, чтобы выучить пошаговые условные распределения в данных)

Если LU подталкивает генератор к созданию реалистичных последовательностей (оцениваемых несовершенным противником), LS дополнительно гарантирует, что он создает реалистичные пошаговые переходы (оцениваемые по истинным целям).

<figure>
<img src="https://raw.githubusercontent.com/sswt/dive2gai/w14/.github/images/w14/timegan2.png" alt="TimeGAN learning scheme" width="75%">
<figcaption align = "center"> Image credit: TimeGAN paper</figcaption>
</figure>

## TimeGCI

[Time-series Generation by Contrastive Imitation (TimeGCI), 2021](https://papers.nips.cc/paper_files/paper/2021/file/f2b4053221961416d47d497814a8064f-Paper.pdf) - ещё одна работа от Van der Schaar лаборатории, но без кода.

Давайте временные ряды назовём траекториями. Предполагается, что есть некий процесс перехода от точки к точке в последовательности. Процесс параметризуется политикой перехода.

Тогда у нас есть два распределения - распределение следующего состояния при условии предыдущих и совместное распределение всех траекторий целиком.

Генеративная модель имеет две цели:

- выучить условное распределение следующего элемента (пошаговый переход в реальных последовательностей)
- точно моделировать совместное распределение реальных последовательностей целиком

С одной стороны, модели авторегрессии, обученные методом MLE, позволяют изучать и вычислять явные распределения переходов, но страдают от накопления ошибок при генерации последовательности (exposure bias). GAN-модели решают эту проблему, но переходы являются неявными и их трудно оценить.

TimeGCI стремится объединить сильные стороны обеих моделей: согласуя моменты для уменьшения накопления ошибки, оптимизируется локальная (но ориентированная на будущее) политика перехода, где сигнал подкрепления обеспечивается глобальной (но пошагово разлагаемой) энергетической моделью, обученной контрастивными методами.

В процессе обучения эти два компонента обучаются совместно, что позволяет избежать нестабильности, характерной для состязательных задач. При инференсе выученная политика служит генератором для итеративного сэмплирования, а выученная энергия - мерой на уровне траектории для оценки качества сэмплирования. Благодаря явному обучению политики, имитирующей последовательное поведение признаков временного ряда, этот подход воплощает в себе «генерацию путем имитации».

## TsT-GAN

[Time-series Transformer Generative Adversarial Networks, 2022](https://arxiv.org/pdf/2205.11164.pdf)


А давайте добавим трансформер.

TsT-GAN состоит из 4-х компонент:

- Embedder-predictor - сеть-трансформер, принимающая на вход реальные многомерные последовательности $x_{1:T}$ и предсказывающая следующий элемент последовательности, чтобы выучить истинное условное распределение текущего токена, при условии предыдущих, вместе с латентным распределением текущего эмбеддинга, обусловленным историей
  - embedder для создания эмбеддингов для каждого элемента последовательности - transformer encoder
  - predictor для генерации следующего токена в последовательности - линейный слой для отображений в исходное пространство
- generator and discriminator
  - генератор берет последовательность случайных векторов $z_1, ..., z_T$ и проецирует их в размерность модели (скрытые эмбеддинги), предиктор используется для преобразования обратно во входное пространство
  - discriminator BERT-подобный, как transformer encoder с двунаправленным вниманием

Параметры предиктора общие между генератором и эмбеддером, но обновляются только при выполнении BPTT через сеть эмбеддер-предиктор, то есть он заморожен на этапе генерации

Функции потерь:

* $L_S$ - supervised loss, штрафует за ошибки предсказания следующего элемента послевательности one step ahead prediction error
* $L_{GAN}$ - GAN adversarial loss, состоит из двух компонент, отдельно для генератора и дискриминатора
* $L_{MM}$ - masked modeling loss для генератора, чтобы помочь $L_{GAN}$, которого недостаточно для выучивания временных зависимостей на длительном периоде
* $L_{ML}$ - вспомогательный moment loss для среднего и дисперсии, вдохновляясь TimeGCI

Модель обучается в три этапа:
- сначала независимо обучаются компоненты эмбеддера и предиктора с использованием целевой функции $L_S$
- затем обучается генератор, используя только целевую функцию маскированного моделирования $L_{MM}$
- заключительный этап включает совместное обучение с использованием всех целевых функций

<figure>
<img src="https://raw.githubusercontent.com/sswt/dive2gai/w14/.github/images/w14/tst-gan.png" alt="TsT-GAN scheme" width="100%">
<figcaption align = "center"> Image credit: TsT-GAN paper</figcaption>
</figure>

Кода нет, но есть код для похожей по метрикам модели COT-GAN.
